In [ ]:
import pandas as pd
import tqdm
import numpy as np
import json

## Load file data

In [ ]:
file_name = '/kaggle/input/mooccubex-dataset/translated_comment.json'

# Reading the JSON file as a list
with open(file_name, 'r') as file:
    data = json.load(file)

In [ ]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Display the first few rows using .head()
df.head()

,id,user_id,text,resource_id,create_time
0,Cm_61,105,Hello teacher,V_13690,1565886314000
1,Cm_74,9520252,The function of this evaluation is too hidden,V_286363,1566559535000
2,Cm_171,11731,is_teacher_join,V_454872,1567700474000
3,Cm_177,11731,Socrates Aristotle Plato,V_454872,1567700497000
4,Cm_180,6875014,test,V_586172,1567700756000


## Loại bỏ giá trị text là NaN

In [ ]:
df = df.dropna(subset=['text'])

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline, AutoTokenizer

## Gán nhãn cho các comment

In [ ]:
# Load the pipeline for sentiment analysis
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment", device=0)

# Get the tokenizer associated with the pipeline
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Assuming your text data is in a column named 'text'
# Create a new column 'sentiment' with the analysis results
# {"NEGATIVE": 0, "NEUTRAL": 1, "POSITIVE": 2}
label_to_num = {"LABEL_0": 0, "LABEL_1": 1, "LABEL_2": 2}

# Áp dụng pipeline và chuyển nhãn thành số
df['sentiment_label'] = df['text'].apply(
    lambda x: label_to_num[pipe(x, truncation=True, max_length=512)[0]['label']]
)
#df_top5['sentiment_score'] = df_top5['text'].apply(
#    lambda x: pipe(x, truncation=True, max_length=512)[0]['score']
#)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
output_path = '/kaggle/working/comment_labeled.csv'
df.to_csv(output_path, index=False)